# Preprocessing

In [1]:
INPUT_PATH = '../../00_EDA/00_v2_11/result/unlabeled.csv'

for FOLD in range(5):
    print('fold = ', FOLD)
    model_list = [    
        {'path':f'../../29_Pseudo4/exp/result/29_v2_01/pseudo_fold{FOLD}.csv', 'w':1}, #deberta-large, seed100, split00_v2_07
        {'path':f'../../29_Pseudo4/exp/result/29_vl_01/pseudo_fold{FOLD}.csv', 'w':1}, #deberta-v3-large, seed100, split00_v2_07
        {'path':f'../../29_Pseudo4/exp/result/29_v2_02/pseudo_fold{FOLD}.csv', 'w':1}, #deberta-xlarge, seed100, split00_v2_07
    ]

    import pandas as pd
    import os
    os.makedirs('result', exist_ok=True)

    w_list = []
    for i,model in enumerate(model_list):
        w = model['w']
        w_list.append(w)

    stack_df = None
    for i,model in enumerate(model_list):
        pred_df = pd.read_csv(model['path'])
        w = w_list[i]

        if 'cat' in model['path']:
            pred_df['pred_ineffective'] = pred_df['Ineffective'] * w / sum(w_list)
            pred_df['pred_adequate'] = pred_df['Adequate'] * w / sum(w_list)
            pred_df['pred_effective'] = pred_df['Effective'] * w / sum(w_list)
        else:
            pred_df['pred_ineffective'] = pred_df['Ineffective'] * w / sum(w_list)
            pred_df['pred_adequate'] = pred_df['Adequate'] * w / sum(w_list)
            pred_df['pred_effective'] = pred_df['Effective'] * w / sum(w_list)
        pred_df = pred_df.rename(columns={
            'pred_ineffective':f'Ineffective_{i}', 
            'pred_adequate':f'Adequate_{i}', 
            'pred_effective':f'Effective_{i}'
        })
        if stack_df is None:
            stack_df = pred_df.copy()
        else:
            stack_df = stack_df.merge(pred_df[['discourse_id',f'Ineffective_{i}',f'Adequate_{i}',f'Effective_{i}']],
                                      on='discourse_id', how='left')

    pseudo_df = pd.read_csv(INPUT_PATH)
    stack_df = pseudo_df[['discourse_id']].merge(stack_df, on='discourse_id', how='left')

    for col_name in ['Ineffective','Adequate','Effective']:
        cols = [f'{col_name}_{i}' for i in range(len(model_list))]
        stack_df[col_name] = stack_df[cols].sum(axis=1)

    stack_df.to_csv(f'result/stack_fold{FOLD}.csv', index=False)
    pseudo_df = pseudo_df[['discourse_id']].merge(stack_df, on='discourse_id', how='left')
    print('pseudo_df.shape = ', pseudo_df.shape)


    # ensemble (simple mean)
    print('ensemble(simple mean) begins...')
    import joblib
    from os.path import join as opj

    for col_name in ['Ineffective','Adequate','Effective']:
        cols = []
        cols += [col for col in pseudo_df.columns if (col.startswith(col_name) and '_' in col)] 
        pseudo_df[col_name] = stack_df[cols].sum(axis=1)
    print('ensemble(simple mean), done')

    pseudo_df.to_csv(f'result/ensemble_pseudo_label_fold{FOLD}.csv', index=False)
    print('pseudo_df.shape = ', pseudo_df.shape)

fold =  0
pseudo_df.shape =  (107528, 20)
ensemble(simple mean) begins...
ensemble(simple mean), done
pseudo_df.shape =  (107528, 20)
fold =  1
pseudo_df.shape =  (107528, 20)
ensemble(simple mean) begins...
ensemble(simple mean), done
pseudo_df.shape =  (107528, 20)
fold =  2
pseudo_df.shape =  (107528, 20)
ensemble(simple mean) begins...
ensemble(simple mean), done
pseudo_df.shape =  (107528, 20)
fold =  3
pseudo_df.shape =  (107528, 20)
ensemble(simple mean) begins...
ensemble(simple mean), done
pseudo_df.shape =  (107528, 20)
fold =  4
pseudo_df.shape =  (107528, 20)
ensemble(simple mean) begins...
ensemble(simple mean), done
pseudo_df.shape =  (107528, 20)


In [2]:
pseudo_df.head(2)

,discourse_id,essay_id,discourse_start,discourse_end,discourse_text,discourse_type,discourse_type_num,predictionstring,Ineffective,Adequate,Effective,Ineffective_0,Adequate_0,Effective_0,Ineffective_1,Adequate_1,Effective_1,Ineffective_2,Adequate_2,Effective_2
0,1.622576e+12,A8445CABFECE,18.0,85.0,Drivers should not be able to use phones while...,Position,Position 1,3 4 5 6 7 8 9 10 11 12 13 14,0.030765,0.949926,0.019308,0.009482,0.319354,0.004497,0.007064,0.321105,0.005164,0.01422,0.309466,0.009647
1,1.622576e+12,A8445CABFECE,86.0,202.0,Drivers who used their phone while operating a...,Claim,Claim 1,15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 3...,0.008580,0.427864,0.563556,0.003430,0.160859,0.169044,0.002120,0.141867,0.189346,0.00303,0.125138,0.205166
